## 11.3 词组的两种组织 集合和序列

机器学习模型如何表示单个单词是个相对没有争议的问题,上一节我们提到了如何进行单词的编码.但是另一个棘手的问题: 如何将这些单词组织成句子?

自然语言中单词的顺序是个非常有趣的问题: 与时间序列不同,句子中的单词没有一个自然典型的前后顺序.不同语言下哪怕都是这些单词,其组合顺序相差很大.即使给定一门语言,通常也可以重新排列单词但是表达的意思完全一致.甚至没有语序,只是一堆乱序的单词,我们也能主动的猜测这句话原本的意思,尽管可能出现很多歧义.自然语言句子的语序和意义相关,但又不那么直接.

如何表达词序在不同的 nlp 架构中千差万别,最直接的方法是抛弃词序,将样本转化为无序的单词集合处理.这种方法可以被称为词袋模型.也可以严格按照单词的顺序进行处理,就像处理时序数据一样(这里可以考虑使用 rnn).混合的方式也是可行的,Transformer 架构整体上是不考虑词序的,但是它将词的位置信息注入了它处理的表征中,这使得 Transformer 能同时查看句子的不同部分(与 rnn 不同).因此 Transformer 和 rnn 也被称作序列模型.

历史上看,机器学习早期在 nlp 上的应用只涉及到词袋模型.随着循环神经网络的再次复兴,人们对序列模型的兴趣在 2015 年才逐渐提高，今天这两种模型依然息息相关。

我们将在 imdb 数据集上演示每种方法,在 4 5 章里我们是使用的是预处理完毕矢量化的词袋数据,接下来我们会直接处理 imdb 的原始文本数据.


### 准备 imdb 原始数据

原始数据集在这里 > <https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz>

解压后得到了 aclImdb 文件夹

```text
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```

- train 是训练数据,test 是测试数据
- pos 是正向数据,neg 是负面数据.
- train/test 各自有 12500,共 25000 条数据.


In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)  #按照种子文件洗牌重排列
    num_val_samples = int(0.2 * len(files))  #取0.2作为验证集
    val_files = files[-num_val_samples:]
    for fname in val_files:  #将验证集移动到单独文件夹
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

将原来的顺序打乱,选取验证集,将验证集移入单独文件夹.


In [8]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [1]:
from tensorflow import keras

batch_size = 32  #批次大小

train_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)  #训练集
val_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)  #验证集
test_ds = keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)  #测试集


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


使用 `text_dataset_from_directory` 构建数据集


In [2]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'this movie let me down decidedly hard. it was a great concept that was ruined with a horrible script. The story just didn\'t flow and was disjointed at best. There were so many elements to this story that were not explained, or were forced into place with out any real thought. elements like the love story could have been expanded on a bit more, and the cannons need to be written in better. the whole main character growing up thing needed more about the training he was receiving and less standing around. everyone likes a good "little guy overcomes" story and this showed promise but with the scripting failures wasn\'t to be. While it did have some pyrotechnics in the final battle sequence it was lackluster due to a lack of choreography. this made for a maddeningly boring watch<br /><br />it could have been so good :(', shape=(), dtype=string)
targets[0]: tf.Tens

至此数据准备完成.


## 词袋模型

最直接处理文本信息的方法,直接忽略顺序,将标记处理成无序的集合.

例子 "the cat sat on the mat" 处理完毕就成了

```text
{"cat", "mat", "on", "sat", "the"}
```

这样做的优点是可以方便的将一句话表示为单一的张量,每一个维度都是一个标记存在不存在.如果使用二进制编码,将一句话表示为一个张量,那么这个张量的维度和集合标记的总数相同,最终的张量中大部分维度都是 0,少数才是 1.(这就是 4 5 章我们使用 imdb 的方式)


In [3]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [4]:
text_vectorization = TextVectorization(
    max_tokens=20000,  #限制最大总标记数量 20000,防止编码空间过大.
    output_mode="binary",  #输出标记为 2进制
)
text_only_train_ds = train_ds.map(lambda x, y: x)  #只有原始文本信息的数据集
text_vectorization.adapt(text_only_train_ds)  # adapt 转换为单词索引表
#张量化数据集
binary_1gram_train_ds = train_ds.map(lambda x, y:
                                     (text_vectorization(x), y))  #训练集
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))  #验证集
binary_1gram_test_ds = test_ds.map(lambda x, y:
                                   (text_vectorization(x), y))  #测试集


使用上一节提到的 TextVectorization 层张量化文本数据.


In [24]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


一个张量有 20000 个维度

每个张量的值完全是 0-1 组成.


In [25]:
from tensorflow import keras
from tensorflow.keras import layers


def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens, ))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)  #全连接层
    x = layers.Dropout(0.5)(x)  #dropout层
    outputs = layers.Dense(1, activation="sigmoid")(x)  #输出层
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

获取 model 的函数,接下来会一直用到.

In [26]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                    save_best_only=True)  #只保存最好模型
]
model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds.cache(),  #验证集
    epochs=10,  #10个epoch
    callbacks=callbacks)


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 10s 16ms/step - loss: 0.3906 - accuracy: 0.8400 - val_loss: 0.3062 - val_accuracy: 0.8762
Epoch 2/10
625/625 [==============================] - 3s 6ms/step - loss: 0.2678 - accuracy: 0.9019 - val_l

In [27]:
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 9s 12ms/step - loss: 0.3006 - accuracy: 0.8789
Test acc: 0.879


我们这里的准确率是 88.7% 相对于基线 50% 已经很好了.


### 二进制编码的双字母组

抛弃词序是非常简化的方法.但是因为一些非常单一的概念是由多个单词表达的,例如 "United States" 如果拆成单个词 "United" 和 "States" 表达的意思将完全不同.因此进行标记拆分文本时,通常会使用 n-gram 而不是直接拆分成单个的词.通常情况下会使用双字母组(2-gram).

还是 "the cat sat on the mat" 的例子,转成 2-gram

```text
{"the", "the cat", "cat", "cat sat", "sat",
 "sat on", "on", "on the", "the mat", "mat"}
```


In [28]:
text_vectorization = TextVectorization(
    ngrams=2,  #配置 n-gram
    max_tokens=20000,
    output_mode="binary",
)

TextVectorization 有 ngrams 参数设置 n-gram.


In [29]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

重新准备数据

In [30]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.keras", save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_6 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 11s 17ms/step - loss: 0.3758 - accuracy: 0.8417 - val_loss: 0.2777 - val_accuracy: 0.8902
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2396 - accuracy: 0.9165 - val_l

In [31]:
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 10s 13ms/step - loss: 0.2662 - accuracy: 0.8979
Test acc: 0.898


测试集精度提高到了 89.6% 提高了 1% 大概.


## TF-IDF 编码的双字母组

我们还可以统计标记出现的次数,来为标记增加一些信息.

还是上面的例子,增加频率统计后:

```text
{"the": 2, "the cat": 1, "cat": 1, "cat sat": 1, "sat": 1,
 "sat on": 1, "on": 1, "on the": 1, "the mat: 1", "mat": 1}
```

如果是文本分类任务,那么标记的次数统计就相当重要了.像是在电影评论中,任何足够长的评论可能都会存在 `terrible` 这个词,但是如果一个评论 `terrible` 出现次数相当多,那这条评论可能会偏向负面评价.


In [32]:
text_vectorization = TextVectorization(
    ngrams=2,  #双字母组
    max_tokens=20000,  #最多20000个标记
    output_mode="count"  #添加词频
)

上面是 TextVectorization 增加词频的方法,设置 `output_mode="count"`.

基于频率的统计有一个天然的缺陷,总有一些词出现的频率远高于其他,但是这些词与特征没什么关系.例如 `a` `is` 之类的,如果蛮干直接上词频统计,这些词会淹没那些与特征密切相关的词.

按照前文的经验,这里可能的方案是数据规范化.将数据减去平均值/方差规范数字.这样的做法在大多数情况下是有意义的,但是文本张量几乎大部分数据都是0,相当稀疏.这样的稀疏性非常有利于计算.但是直接上前面的归一化会破坏这样的稀疏性,增加计算的成本.因此我们能够在文本张量归一化使用的运算只有除法而已.

文本张量归一化只能用除法,那么分母呢?最好的方法是 TF-IDF(term frequency, inverse document frequency) 规范化的方法.


### 理解 TF-IDF

关于文本数据有两个有意思的点

- 一个给定的术语在文档内重复次数越多,这个词对理解文档越重要.
- 几乎所有文档都存在这类似 'is' 'the' 这样出现频率很高,但是对理解文档没什么作用的单词.

tf-idf 融合了上面两种观点

- tf = term frequency,单词在文档的出现次数
- idf = inverse document frequency,逆文档频率,需要一个语料库,单词越常见 idf 越小.

最后 tf-idf = tf * idf.

```py
def tfidf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset) + 1)
    return term_freq / doc_freq
```

这里是除也没问题,把 idf 计算公式颠倒即可.


In [34]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf-idf",
)

当然 tf-idf 这样常用的方法,已经内置在了 `TextVectorization` 层了,`output_mode` 参数设置为 `tf-idf`.

In [35]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

In [36]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras", save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 20000)]           0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                320016    
_________________________________________________________________
dropout_4 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 12s 19ms/step - loss: 0.5035 - accuracy: 0.7766 - val_loss: 0.3008 - val_accuracy: 0.8882
Epoch 2/10
625/625 [==============================] - 4s 6ms/step - loss: 0.3282 - accuracy: 0.8597 - val_l

In [37]:
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 11s 13ms/step - loss: 0.3002 - accuracy: 0.8898
Test acc: 0.890


89.0% 似乎并没有特别的帮助,但是普遍的 tf-idf 能够比双字母组编码要再提高 1%.


### 导出可以处理原始字符串的模型

上面我们是将文本处理部分完全放在了模型外部,如果我们想导出一个在生产环境运行的模型,我们必须确保模型可以处理原始字符串,这在 keras 中很容易.


In [38]:
inputs = keras.Input(shape=(1, ), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

还是搭积木一样,将定义的 text_vectorization 串入模型.


In [41]:
import tensorflow as tf

raw_text_data = tf.convert_to_tensor([
    ["That was an excellent movie, I loved it."],
])  #原始数据
predictions = inference_model(raw_text_data)  #预测
print(f"{float(predictions[0] * 100):.2f} percent positive")

93.57 percent positive


## 序列模型

上一章我们了解了对时序数据,没了时间顺序,模型会存在很大瓶颈.在文本数据中如果基于人工的进行特征筛选,模型的准确度能有很好的提升.但是请记住,深度学习的历史就是摆脱人工特征工程的历史,让模型接触原始数据,自行学习数据背后的特征.序列模型就是输入原始的单词序列,模型自行找出特征.

实现序列模型

- 将输入样本转换为整数序列(一个整数代表一个词)
- 将每个整数映射到一个张量,获得张量序列.
- 将张量序列送入可以从顺序中提取特征的模型,这里可以是 一维cnn rnn 或者 Transformer.

2016-2017 年的那段时间里,双向 rnn (特别是双向 lstm)被认为是最先进的序列模型.我们第一个序列模型的例子就是双向 lstm.但是限制序列模型使用的几乎都是 Transformer,我们将在下面内容涉及.非常奇怪的是一维卷积在 nlp 从来没有流行过,尽管根据我自己的经验,一个深度可分离的一维 cnn 往往能达到和 双向lstm 同等的性能,而且计算成本大大降低.


In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

max_length = 600  #前600字
max_tokens = 20000
text_vectorization = TextVectorization(
    max_tokens=max_tokens,  #取词频前 20000的词
    output_mode="int",  #返回整数序列
    output_sequence_length=max_length,  #前600字以后截断输入
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y))  #数据集
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y))
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y))

我们这里限制了过长的评论数据,限制一个评论最多 600 词,imdb 评论的平均长度是 233 词,大于 600 词的仅有 5%.


In [6]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)  #整数编码为 20000 维度向量
x = layers.Bidirectional(layers.LSTM(32))(embedded)  #bidirectional LSTM
x = layers.Dropout(0.5)(x)  #dropout
outputs = layers.Dense(1, activation="sigmoid")(x)  #最终二分类的输出
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
tf.one_hot (TFOpLambda)      (None, None, 20000)       0         
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                5128448   
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
_________________________________________________________________


我们对整数序列采用了 one-hot 编码,之后模型添加了一个简单的双向 ltsm.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=10,
          batch_size=1,
          callbacks=callbacks)


**前排提醒**

- 这一步对显存要求很高,batch_size=1 改成 1 也难通过...
- 即使使用 cpu 训练,也异常的慢

最好在 colab 测试.这里本机没法跑了...


In [ ]:
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

根据作者的数据是 87% 还不如前面的模型...

这里的输入相当大 600*20000 = 12000000 个浮点数,直接 cpu 跑都能占到 9G 内存,别提那点显存了,根本不够.

明显将单词转换成张量这样编码非常简单,但远远不是最佳做法.还有更好的做法 -> 单词嵌入(词嵌入,第一版这样翻译的)


### 词嵌入

在做上文的 one-hot 编码时,实际上是在做一个特征工程,而且我们带入了一个关于特征空间的假设前提: 编码的不同标记直接互相独立.one-hot 向量之间是彼此正交的,但是对应的单词却并非没有关系.像是 'movie' 和 'film' 这样的同义词,基本可以互换.因此表示 'movie' 和 'film' 的应该是同一向量或者两个非常接近的向量.

更抽象一点,两个词向量之间的几何关系应当能够反映这些词之间的语义关系.例如: 一个合理的词向量空间,同义词向量之间的几何距离非常接近,或者就是同一向量.相关词之间的几何距离应该是接近,无关词之间的几何距离应该很远.

one-hot 编码的向量 与 词嵌入得到的向量 异同

- one-hot 的向量是二进制的,非常稀疏(大多数都是 0).词嵌入得到的向量则是密集浮点向量.
- one-hot 编码的向量维度惊人,上面的例子中是 20000,而词嵌入向量维度处理大的词汇表时也就是 256 512 或 1024 维.

词嵌入得到的向量是将更多的信息装入了更少的维度.

![word_embeddings](word_embeddings.png)


除了更加密集外,词嵌入也会是结构化的,相似的词会嵌入到相近的位置,这也让嵌入空间的特定方向有了意义.

![toy_word_embedding_space](toy_word_embedding_space.png)

四个词被嵌入了二维平面,猫 狗 狼 老虎.我们在这个二维平面上取向量

- 向量是自下而上,这个向量可以让我们取到 猫->虎 或者 狗->狼.这个向量可以解释为: 宠物到野生动物.
- 向量自左向右,可以得到 狗->猫 狼->虎.这个向量可以解释为: 从犬科到猫科.

在现实世界的词嵌入空间中,有意义的几何变换常见的是性别和复数.国王+女性->女王,国王+复数->国王们(kings).这样的词嵌入的空间可能有成千上万的解释.


获取词嵌入有两种方式

- 进行主任务(文本分类或情感预测)的同时获取词嵌入.这种情况下,一开始是随机的词向量.然后对这些词向量进行学习,学习的方式与神经网络学习方式相同(都是梯度下降,学习权重).
- 将预先计算好的词嵌入加载到模型,这种做法称为预训练嵌入.

接下来我们都会用到.


#### Embedding 层学习词嵌入

是否存在一个理想的词嵌入空间,可以完美映射人类语言,并可用于任何 nlp 任务?可能存在,但是目前还没有.另外也不存在人类语言这样的概念,各种不同的自然语言之间非常不同,语言是特定文化和特定环境的映射.从更实际的角度,一个好的词嵌入空间很大程度上取决于不同 的任务.对英文电影评论完美的词嵌入空间,可能完全不同于对英文法律文档完美的词嵌入空间.这中间涉及到很多词在不同场景下多意和语义在不同任务下重要性完全不同.

因此在每个任务都学习生成新的嵌入空间是合理的,幸运的是通过反向传播让这种学习非常简单.当然在 kaeras 中更简单,直接加入 `Embedding` 层.


In [5]:
import tensorflow.keras.layers as layers
max_tokens = 20000

In [6]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

Embedding 可以直接理解为一个字典,将整数序列映射到一个密集向量.接受整数输入,返回相关的向量.实际上被看作是一个字典查询.

Embedding 的输入是二维整数张量,形状是 (batch_size, sequence_length）,其中每个条目是一个整数序列,最终该层返回一个三维浮点张量,形状是(batch_size, sequence_length, embedding_dimensionality)

实例化一个 Embedding 层后,其权重和其他神经网络一样都是随机的,在训练过程中通过反向传播逐渐调整,将词嵌入空间逐渐转换为下游模型可以使用的东西.完成训练后词嵌入空间会现实很多针对问题的特定结构.


In [9]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens,
                            output_dim=256)(inputs)  #词嵌入层
x = layers.Bidirectional(layers.LSTM(32))(embedded)  #双向 lstm
x = layers.Dropout(0.5)(x)  #dropout
outputs = layers.Dense(1, activation="sigmoid")(x)  #最终二分类的输出
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 256)         5120000   
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                73984     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________


In [10]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
                                    save_best_only=True)  #只保存最佳模型
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 49s 71ms/step - loss: 0.4884 - accuracy: 0.7786 - val_loss: 0.3700 - val_accuracy: 0.8486
Epoch 2/10
625/625 [==============================] - 40s 64ms/step - loss: 0.3198 - accuracy: 0.8841 - val_loss: 0.3133 - val_accuracy: 0.8782
Epoch 3/10
625/625 [==============================] - 41s 66ms/step - loss: 0.2419 - accuracy: 0.9162 - val_loss: 0.3295 - val_accuracy: 0.8698
Epoch 4/10
625/625 [==============================] - 41s 65ms/step - loss: 0.2052 - accuracy: 0.9287 - val_loss: 0.4053 - val_accuracy: 0.8320
Epoch 5/10
625/625 [==============================] - 41s 66ms/step - loss: 0.1718 - accuracy: 0.9420 - val_loss: 0.3404 - val_accuracy: 0.8856
Epoch 6/10
625/625 [==============================] - 41s 66ms/step - loss: 0.1460 - accuracy: 0.9524 - val_loss: 0.3672 - val_accuracy: 0.8820
Epoch 7/10
625/625 [==============================] - 39s 62ms/step - loss: 0.1156 - accuracy: 0.9612 - val_loss: 0.4143 - val_accuracy:

In [11]:
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 39s 49ms/step - loss: 0.3534 - accuracy: 0.8550
Test acc: 0.855


训练本身快多了,也没有爆显存.毕竟只有 256 维度向量而不是 20000.

测试结果 85.5% 尚可,但是和 n-gram 还有差距.部分原因是 n-gram 使用的是完整评论,而这里仅仅是前 600 词.


#### 填充和跳过

有一件事情影响力影响力模型性能,我们的输入序列中充满了 0,在 TextVectorization 使用的参数使得长于 600 的词被舍弃,少于 600 词的句子又被填充 0 到 600.这样填充的数据是无意义的,只是一堆 0.

我们使用的双向 rnn,两个 rnn 层并行运行,一个按照正序,一个按照逆序.如果原句很短,会填充大量的 0 直到 600 词,这样在正序的 rnn 层会进行几百次无意义的迭代,这样的迭代有可能使得 rnn 学习到的有意义的模式消失.

我们需要一些方法告诉模型跳过那些迭代,以避开在无意义的填充数据上迭代.这个 api 是 `masking`.


Embedding 层会生成一个与输入数据对应的 `mask`,这个掩码是 1/0 的张量,形状是(batch_size, sequence_length).mask[i, t] 表示的对应的样本 i 的时间段 t 是否应该被跳过.


In [14]:
embedding_layer = layers.Embedding(input_dim=10, output_dim=256, mask_zero=True)

In [16]:
some_input = [[4, 3, 2, 1, 0, 0, 0], [5, 4, 3, 2, 1, 0, 0],
              [2, 1, 0, 0, 0, 0, 0]]
mask = embedding_layer.compute_mask(some_input)
mask

<tf.Tensor: shape=(3, 7), dtype=bool, numpy=
array([[ True,  True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False, False],
       [ True,  True, False, False, False, False, False]])>

默认是不会生成 `mask`,可以通过设置 `mask_zero = True` 激活.最后调用 `compute_mask` 获取 `mask` 结果.

实践中,几乎没有手动管理 mask 的情况,keras 会将其和序列自动传送给能处理它们每一层.rnn 接收了 mask 会跳过填充数据的迭代,如果最后模型返回一个完整序列,那么损失函数计算也将跳过这些填充数据.


In [17]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens,
                            output_dim=256,
                            mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 256)         5120000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                73984     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________________________


In [18]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 51s 74ms/step - loss: 0.3895 - accuracy: 0.8268 - val_loss: 0.2778 - val_accuracy: 0.8872
Epoch 2/10
625/625 [==============================] - 44s 70ms/step - loss: 0.2249 - accuracy: 0.9158 - val_loss: 0.2740 - val_accuracy: 0.8878
Epoch 3/10
625/625 [==============================] - 44s 70ms/step - loss: 0.1622 - accuracy: 0.9416 - val_loss: 0.3131 - val_accuracy: 0.8808
Epoch 4/10
625/625 [==============================] - 45s 72ms/step - loss: 0.1211 - accuracy: 0.9579 - val_loss: 0.3482 - val_accuracy: 0.8764
Epoch 5/10
625/625 [==============================] - 43s 68ms/step - loss: 0.0924 - accuracy: 0.9664 - val_loss: 0.3455 - val_accuracy: 0.8824
Epoch 6/10
625/625 [==============================] - 42s 67ms/step - loss: 0.0642 - accuracy: 0.9776 - val_loss: 0.4051 - val_accuracy: 0.8818
Epoch 7/10
625/625 [==============================] - 41s 65ms/step - loss: 0.0484 - accuracy: 0.9843 - val_loss: 0.4275 - val_accuracy:

In [19]:
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 21s 26ms/step - loss: 0.2911 - accuracy: 0.8811
Test acc: 0.881


居然能到 88.1% 这次和书上差不太多了,提升很实在.


### 预处理模型

直接训练 Embedding 层有一个限制,样本量要足够训练才行,如果训练数据非常少,以至于无法通过训练得到可用 Embedding 层.和图像处理那里一样,我们使用别人已经处理过的 Embedding 层.

这种词嵌入通常是使用词频同居计算得出的,所使用的技术非常庞杂,有些涉及神经网络,有些则不是.bengio 首先在 21 世纪初研究了一种思路,使用无监督方法计算一个密集的低维词嵌入空间,但直到最著名且成功的词嵌入方案之一 word2vec 算法发布后,这一思路才开始在研究领域和工业英语取得成功.word2ver 算法是 google 的 tomas mikolov 于 2013 年发布,其维度抓取特定的语义属性,比如性别.

有许多预处理的词嵌入数据库,都可以用在 keras 的 Embedding 层,常用的另外一个是 Glove (Global Vectors for Word Representation) 是斯坦福大学于 2014 年开发,基于对词共现统计矩阵进行因式分解.开发者已经公开了数百万个英文标记的预处理嵌入,大都是维基百科和 Common Crawl 数据.

我们这里会使用 GloVe 嵌入 keras,word2ver 等的嵌入也完全相同.


#### 下载 GloVe


In [20]:
# colab 运行
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

In [ ]:
下载数据,解压数据

In [22]:
import numpy as np

path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file,
          encoding='UTF-8') as f:  #中文系统注意,要加上 encoding='UTF-8'
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


这里选取了 `glove.6B.100d.txt` 建立单词索引.

#### 在模型中嵌入 glove


In [23]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()  #获取词汇表
word_index = dict(zip(vocabulary, range(len(vocabulary))))  #将词汇表转换成索引

embedding_matrix = np.zeros((max_tokens, embedding_dim))  #初始化词嵌入矩阵
for word, i in word_index.items():
    if i < max_tokens:  # i < 20000 词频在前 20000
        embedding_vector = embeddings_index.get(word)  #获取词向量
    if embedding_vector is not None:  #如果词汇表中的词在词嵌入矩阵中有对应的值
        embedding_matrix[i] = embedding_vector

我们建立了嵌入矩阵填充,然后将嵌入矩阵加载到 Embedding 层.

嵌入矩阵形状必须是 (max_words, embedding_dim),其中每个条目 i 必须包含参考词的索引中索引 i
的词对应的 embedding_dim 维度向量.索引 0 不表示任何单词或标记,只是占位符.

In [24]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,  #冻结
    mask_zero=True,
)

需要冻结 Embedding 层.

In [25]:
inputs = keras.Input(shape=(None, ), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 100)         2000000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                34048     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
_________________________________________________________________


这里模型有一些变化, Embedding 层是 100 维预训练的 glove.

In [26]:
callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds,
          validation_data=int_val_ds,
          epochs=10,
          callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 35s 49ms/step - loss: 0.5625 - accuracy: 0.7046 - val_loss: 0.4526 - val_accuracy: 0.7912
Epoch 2/10
625/625 [==============================] - 30s 47ms/step - loss: 0.4467 - accuracy: 0.7969 - val_loss: 0.6346 - val_accuracy: 0.7150
Epoch 3/10
625/625 [==============================] - 29s 46ms/step - loss: 0.3965 - accuracy: 0.8256 - val_loss: 0.3743 - val_accuracy: 0.8378
Epoch 4/10
625/625 [==============================] - 29s 47ms/step - loss: 0.3621 - accuracy: 0.8447 - val_loss: 0.3908 - val_accuracy: 0.8308
Epoch 5/10
625/625 [==============================] - 29s 47ms/step - loss: 0.3345 - accuracy: 0.8582 - val_loss: 0.4015 - val_accuracy: 0.8214
Epoch 6/10
625/625 [==============================] - 29s 46ms/step - loss: 0.3114 - accuracy: 0.8715 - val_loss: 0.3273 - val_accuracy: 0.8652
Epoch 7/10
625/625 [==============================] - 29s 46ms/step - loss: 0.2949 - accuracy: 0.8787 - val_loss: 0.3462 - val_accuracy:

In [27]:
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 19s 22ms/step - loss: 0.2954 - accuracy: 0.8757
Test acc: 0.876


效果和上面的结果大差不差.这是因为这里的样本已经很多,都足以直接训练出可用的 Embedding 层了,所以使用预处理模型,收益不大.但是假设向第一版的例子,只使用非常少了训练数据做分类.效果会非常显著.
